<a href="https://colab.research.google.com/github/viniciusrpb/cic0269_natural_language_processing/blob/main/cap07_1_probabilistic_language_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capítulo 7 - Modelos de Linguagem Probabilísticos

Um modelo de linguagem calcula a probabilidade de obtermos uma sentença ou uma sequência de palavras.

Neste capítulo, vamos abordar os modelos de linguagem probabilísticos baseados em N-gramas.

Vamos considerar o pequeno *corpus* criado nas aulas anteriores:

In [1]:
corpus = []
corpus.append('Batatinha quando nasce esparrama pelo chão igual batatinha')
corpus.append('A pior experiência da minha vida')
corpus.append('Quero meu dinheiro de volta pois é meu e é meu')
corpus.append('A experiência do dinheiro esparrama minha vida')
corpus.append('Acai é a melhor coisa da vida')

## 7.1 Unigramas

Vamos criar um Bag-of-Words (BoW) tradicional baseado em contagem para utilizarmos na construção de um modelo de linguagem unigrama.

Ao mesmo tempo, vamos coletar a quantidade total de termos $n$ de todo o corpus:

In [2]:
unigram = {}
n=0
for i,doc in enumerate(corpus):
    unigram['sentenca {}'.format(i+1)] = dict()
    tokens = doc.split()
    for word in tokens:
        if word in unigram['sentenca {}'.format(i+1)]:
            unigram['sentenca {}'.format(i+1)][word] += 1
        else:
            unigram['sentenca {}'.format(i+1)][word] = 1
        n+=1

unigram

{'sentenca 1': {'Batatinha': 1,
  'batatinha': 1,
  'chão': 1,
  'esparrama': 1,
  'igual': 1,
  'nasce': 1,
  'pelo': 1,
  'quando': 1},
 'sentenca 2': {'A': 1,
  'da': 1,
  'experiência': 1,
  'minha': 1,
  'pior': 1,
  'vida': 1},
 'sentenca 3': {'Quero': 1,
  'de': 1,
  'dinheiro': 1,
  'e': 1,
  'meu': 3,
  'pois': 1,
  'volta': 1,
  'é': 2},
 'sentenca 4': {'A': 1,
  'dinheiro': 1,
  'do': 1,
  'esparrama': 1,
  'experiência': 1,
  'minha': 1,
  'vida': 1},
 'sentenca 5': {'Acai': 1,
  'a': 1,
  'coisa': 1,
  'da': 1,
  'melhor': 1,
  'vida': 1,
  'é': 1}}

## 7.2. Bigramas

De maneira similar, podemos calcular os bigramas. Para isso, seja uma sentença $W$ que em sua forma tokenizada é descrita como $\{w_1, w_2, \ldots, w_n\}$.

O Bag-of-Words de bigramas é construído pegando cada par consecutivo de palavras $(w_1,w_2), (w_2,w_3), \ldots, (w_{n-1},w_n)$. Vamos construir um Bag-of-Words de bigrams para facilitar o entendimento.

**Atenção:** não se trata de uma permutação entre todas as palavras do *corpus*!

Vamos criar um dicionário em que cada bigrama é uma chave sob a forma de uma tupla:

In [3]:
bigram = {}

for i,doc in enumerate(corpus):
    bigram['sentenca {}'.format(i+1)] = dict()
    tokens = doc.split()
    for j in range(0,len(tokens)-1):
        par = (tokens[j],tokens[j+1])
        if par in bigram['sentenca {}'.format(i+1)]:
            bigram['sentenca {}'.format(i+1)][par] += 1
        else:
            bigram['sentenca {}'.format(i+1)][par] = 1

bigram

{'sentenca 1': {('Batatinha', 'quando'): 1,
  ('chão', 'igual'): 1,
  ('esparrama', 'pelo'): 1,
  ('igual', 'batatinha'): 1,
  ('nasce', 'esparrama'): 1,
  ('pelo', 'chão'): 1,
  ('quando', 'nasce'): 1},
 'sentenca 2': {('A', 'pior'): 1,
  ('da', 'minha'): 1,
  ('experiência', 'da'): 1,
  ('minha', 'vida'): 1,
  ('pior', 'experiência'): 1},
 'sentenca 3': {('Quero', 'meu'): 1,
  ('de', 'volta'): 1,
  ('dinheiro', 'de'): 1,
  ('e', 'é'): 1,
  ('meu', 'dinheiro'): 1,
  ('meu', 'e'): 1,
  ('pois', 'é'): 1,
  ('volta', 'pois'): 1,
  ('é', 'meu'): 2},
 'sentenca 4': {('A', 'experiência'): 1,
  ('dinheiro', 'esparrama'): 1,
  ('do', 'dinheiro'): 1,
  ('esparrama', 'minha'): 1,
  ('experiência', 'do'): 1,
  ('minha', 'vida'): 1},
 'sentenca 5': {('Acai', 'é'): 1,
  ('a', 'melhor'): 1,
  ('coisa', 'da'): 1,
  ('da', 'vida'): 1,
  ('melhor', 'coisa'): 1,
  ('é', 'a'): 1}}

## 7.3. Modelo de Linguagem baseado em Bigramas

Queremos calcular a probabilidade de obtermos uma sentença $W$, isto é, $P(W) = P(w_1,w_2,\ldots,w_n)$.

Considerando que essas probabilidades são calculadas com base em um vocabulário "gigante" e representativo, desenvolvemos conforme a Regra da Cadeia:

\begin{equation}
P(w_1,w_2,w_3,\ldots,w_n) = P(w_1) \times P(w_2 | w_1) \times P(w_3 | w_1,w_2) \times \ldots P(w_n | w_1,w_2, \ldots, w_{n-1})
\end{equation}

Para *corpus* grandes e com exemplos contendo muitas palavras, o cálculo acima é inviável. Por isso, utilizamos a suposição de Andrei Markov em que pegamos apenas os $k-1$ termos já processados:

\begin{equation}
P(w_i | w_1, w_2, \ldots,w_{i-1}) \approx P(w_i|w_1, \ldots , w_{i-k})
\end{equation}

em que se $k=1$, temos a aproximação para o modelo Unigrama:

\begin{equation}
P(w_i | w_1, w_2, \ldots,w_{i-1}) \approx P(w_i)
\end{equation}

e, por sua vez, se $k=2$, temos a aproximação para o modelo Bigrama:

\begin{equation}
P(w_i | w_1, w_2, \ldots,w_{i-1}) \approx P(w_i| w_{i-1})
\end{equation}

### Exemplo

Se $k=2$, para calcularmos a probabilidade de obtermos a frase "melhor coisa da vida" no modelo de linguagem Bigrama, devemos calcular o seguinte produtório:

$P(``\textrm{melhor coisa da vida''}) = P(``\textrm{melhor''},``\textrm{coisa''},``\textrm{da''},``\textrm{vida''}) = $

$= P(``\textrm{melhor''}) \times P(``\textrm{coisa''}|``\textrm{melhor''}) \times P(``\textrm{da''}|``\textrm{melhor''}) \times P(``\textrm{vida''}|``\textrm{da''})$

### Como calcular as probabilidades?

A principal informação que temos em mãos é a contagem de termos no *corpus*, o que depende do tipo de N-grama utilizado.

No Unigrama, as probabilidades são calculadas como a frequência de cada palavra normalizada pela quantidade total de palavras do vocabulário:

\begin{equation}
P(w_i) = \frac{count(w_i)}{N}
\end{equation}

Já no bigrama, precisamos contar a quantidade de ocorrências de cada par de palavras (bigrama) $(w_{i},w_{i-1})$:

\begin{equation}
P(w_i|w_{i-1}) = \frac{count(w_i,w_{i-1})}{count(w_i)}
\end{equation}

Vamos representar as funções *count* pelos seguintes histogramas de unigramas e de bigramas. O código-fonte para calcular esses histogramas é fornecido abaixo:

In [4]:
histograma_unigram = {}
histograma_bigram = {}

In [5]:
for doc in bigram:
    for par in bigram[doc]:
        if par in histograma_bigram:
            histograma_bigram[par] += bigram[doc][par]
        else:
            histograma_bigram[par] = bigram[doc][par]

In [6]:
histograma_bigram

{('A', 'experiência'): 1,
 ('A', 'pior'): 1,
 ('Acai', 'é'): 1,
 ('Batatinha', 'quando'): 1,
 ('Quero', 'meu'): 1,
 ('a', 'melhor'): 1,
 ('chão', 'igual'): 1,
 ('coisa', 'da'): 1,
 ('da', 'minha'): 1,
 ('da', 'vida'): 1,
 ('de', 'volta'): 1,
 ('dinheiro', 'de'): 1,
 ('dinheiro', 'esparrama'): 1,
 ('do', 'dinheiro'): 1,
 ('e', 'é'): 1,
 ('esparrama', 'minha'): 1,
 ('esparrama', 'pelo'): 1,
 ('experiência', 'da'): 1,
 ('experiência', 'do'): 1,
 ('igual', 'batatinha'): 1,
 ('melhor', 'coisa'): 1,
 ('meu', 'dinheiro'): 1,
 ('meu', 'e'): 1,
 ('minha', 'vida'): 2,
 ('nasce', 'esparrama'): 1,
 ('pelo', 'chão'): 1,
 ('pior', 'experiência'): 1,
 ('pois', 'é'): 1,
 ('quando', 'nasce'): 1,
 ('volta', 'pois'): 1,
 ('é', 'a'): 1,
 ('é', 'meu'): 2}

In [7]:
for doc in unigram:
    for par in unigram[doc]:
        if par in histograma_unigram:
            histograma_unigram[par] += unigram[doc][par]
        else:
            histograma_unigram[par] = unigram[doc][par]

In [8]:
histograma_unigram

{'A': 2,
 'Acai': 1,
 'Batatinha': 1,
 'Quero': 1,
 'a': 1,
 'batatinha': 1,
 'chão': 1,
 'coisa': 1,
 'da': 2,
 'de': 1,
 'dinheiro': 2,
 'do': 1,
 'e': 1,
 'esparrama': 2,
 'experiência': 2,
 'igual': 1,
 'melhor': 1,
 'meu': 3,
 'minha': 2,
 'nasce': 1,
 'pelo': 1,
 'pior': 1,
 'pois': 1,
 'quando': 1,
 'vida': 3,
 'volta': 1,
 'é': 3}

Agora vamos criar uma função que calcula a probabilidade de obtermos uma determinada sentença qualquer de "teste".

**Não finalizado! Falta tratar o log no produtório e casos de probabilidade zero. Continuação na aula do dia 07/07/2022**

In [9]:
def prob(sentence):

    tokens = sentence.split()

    p = histograma_unigram[tokens[0]]/n

    for i in range(0,len(tokens)-1):
        p = p*(histograma_bigram[(tokens[i],tokens[i+1])]/histograma_unigram[tokens[i+1]])
    
    return p

Sentença de teste

In [10]:
prob("Batatinha")

0.02564102564102564